In [1]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [2]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [3]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [4]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,thresh):
    return 100 * n + thresh

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def newThresh():
    global n
    global thresh
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    # calculate new threshold
    thresh = nthThresh(n, thresh)
    # add one to threshold n value
    n += 1

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(trans, death):
    note = random.choice(midiNotes)
    return note + trans - death

In [5]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = []
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 60, 62, 63, 65, 67, 68, 70, 74, 77, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [6]:
times.append(incubate(n))

while len(times) > 0:
    
    # determine how much time has elapse since beginning of piece
    now = min(times)
    
    # if symptomatic add case info to weathering
    if decide(40) == 0:

        # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
        sympCount += 1

         # if case count is at or over threshold number
        if sympCount > nthThresh(n, thresh):
            newThresh()

            
        weathering['time'].append(now)

        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        # select which note
        weathering['note'].append(midi(0, death))

        weathering['n'].append(n)
    
   

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # determine time elapsed since beginning and append to times list
        time = now + delay
        times.append(time)

    # remove current case from timestamps of remaining cases
    times.pop(times.index(now))


In [7]:
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n'])))
# result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n']))

len(result)


11506

In [8]:
result

[(78000.0, 3, 60, 0, 1),
 (86666.66666666667, 2, 77, 0, 1),
 (91333.33333333333, 0, 63, 0, 1),
 (94666.66666666667, 1, 68, 0, 1),
 (104666.66666666666, 5, 62, 0, 1),
 (110000.0, 5, 58, 0, 1),
 (112666.66666666666, 0, 48, 0, 1),
 (117333.33333333334, 2, 51, 0, 1),
 (117999.99999999999, 5, 62, 0, 1),
 (121333.33333333334, 0, 70, 0, 1),
 (130000.0, 5, 68, 0, 1),
 (130000.00000000001, 3, 60, 0, 1),
 (136000.0, 1, 65, 0, 1),
 (136000.0, 3, 70, 0, 1),
 (136666.66666666666, 2, 48, 0, 1),
 (138000.0, 0, 58, 0, 1),
 (139333.33333333334, 0, 63, 0, 1),
 (142666.66666666666, 1, 63, 0, 1),
 (144000.0, 5, 51, 0, 1),
 (145333.33333333334, 5, 55, 0, 1),
 (146000.0, 2, 74, 0, 1),
 (147333.3333333333, 3, 58, 0, 1),
 (148000.0, 0, 55, 0, 1),
 (148000.0, 1, 68, 0, 1),
 (148000.0, 1, 70, 0, 1),
 (148000.0, 2, 70, 0, 1),
 (148000.0, 3, 63, 0, 1),
 (149333.3333333333, 5, 74, 0, 1),
 (149333.33333333334, 3, 60, 0, 1),
 (150000.0, 2, 80, 0, 1),
 (152000.0, 1, 51, 0, 1),
 (154666.66666666666, 5, 80, 0, 1),
 (15

In [9]:
# for i, r in enumerate(result):
#     print(f'{i}: {r[4]}')



In [10]:
meep = round(result[1][0],6)
moop = round(result[2][0],6)
print(meep)
print(moop)


86666.666667
91333.333333


In [11]:
timing = {}
ms = 0
infections = -1

for case in result:
    
    time = round(case[0], 6)

    if time != ms:
        infections += 1
        timing[infections] = {'demoNotes': [case[1], case[2], case[3]]}
        timing[infections]['delay'] = round(time - ms, 6)
        ms = time

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(case[i])


In [12]:
timing



{0: {'demoNotes': [3, 60, 0], 'delay': 78000.0},
 1: {'demoNotes': [2, 77, 0], 'delay': 8666.666667},
 2: {'demoNotes': [0, 63, 0], 'delay': 4666.666666},
 3: {'demoNotes': [1, 68, 0], 'delay': 3333.333334},
 4: {'demoNotes': [5, 62, 0], 'delay': 10000.0},
 5: {'demoNotes': [5, 58, 0], 'delay': 5333.333333},
 6: {'demoNotes': [0, 48, 0], 'delay': 2666.666667},
 7: {'demoNotes': [2, 51, 0], 'delay': 4666.666666},
 8: {'demoNotes': [5, 62, 0], 'delay': 666.666667},
 9: {'demoNotes': [0, 70, 0], 'delay': 3333.333333},
 10: {'demoNotes': [5, 68, 0, 3, 60, 0], 'delay': 8666.666667},
 11: {'demoNotes': [1, 65, 0, 3, 70, 0], 'delay': 6000.0},
 12: {'demoNotes': [2, 48, 0], 'delay': 666.666667},
 13: {'demoNotes': [0, 58, 0], 'delay': 1333.333333},
 14: {'demoNotes': [0, 63, 0], 'delay': 1333.333333},
 15: {'demoNotes': [1, 63, 0], 'delay': 3333.333334},
 16: {'demoNotes': [5, 51, 0], 'delay': 1333.333333},
 17: {'demoNotes': [5, 55, 0], 'delay': 1333.333333},
 18: {'demoNotes': [2, 74, 0], 'd

In [13]:
with open("../resources/weathering.json", "w") as outfile:
    json.dump(timing, outfile)
    